In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.3.1


In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1)  # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1)  # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10)  # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10)  # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_deep'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.8310, train_loss: 2.05389595, train_accuracy: 0.6094, test_accuracy: 0.3628
Epoch: [ 0] [    1/  468] time: 1.3580, train_loss: 1.94993579, train_accuracy: 0.5625, test_accuracy: 0.5030
Epoch: [ 0] [    2/  468] time: 1.8400, train_loss: 1.68026173, train_accuracy: 0.5781, test_accuracy: 0.5009
Epoch: [ 0] [    3/  468] time: 2.2275, train_loss: 1.54057360, train_accuracy: 0.6328, test_accuracy: 0.6139
Epoch: [ 0] [    4/  468] time: 2.6075, train_loss: 1.31307936, train_accuracy: 0.7031, test_accuracy: 0.6928
Epoch: [ 0] [    5/  468] time: 2.9965, train_loss: 1.04494917, train_accuracy: 0.7656, test_accuracy: 0.7041
Epoch: [ 0] [    6/  468] time: 3.3525, train_loss: 0.88658726, train_accuracy: 0.7578, test_accuracy: 0.7447
Epoch: [ 0] [    7/  468] time: 3.6957, train_loss: 0.68156606, train_accuracy: 0.7969, test_accuracy: 0.7458
Epoch: [ 0] [    8/  468] time: 4.0669,

Epoch: [ 0] [   74/  468] time: 26.9187, train_loss: 0.15899055, train_accuracy: 0.9609, test_accuracy: 0.9173
Epoch: [ 0] [   75/  468] time: 27.2787, train_loss: 0.29135191, train_accuracy: 0.8984, test_accuracy: 0.9183
Epoch: [ 0] [   76/  468] time: 27.7123, train_loss: 0.16332662, train_accuracy: 0.9375, test_accuracy: 0.9234
Epoch: [ 0] [   77/  468] time: 28.0503, train_loss: 0.13821095, train_accuracy: 0.9609, test_accuracy: 0.9307
Epoch: [ 0] [   78/  468] time: 28.3933, train_loss: 0.24330409, train_accuracy: 0.9297, test_accuracy: 0.9329
Epoch: [ 0] [   79/  468] time: 28.7372, train_loss: 0.29685283, train_accuracy: 0.9375, test_accuracy: 0.9316
Epoch: [ 0] [   80/  468] time: 29.1561, train_loss: 0.21975699, train_accuracy: 0.9297, test_accuracy: 0.9289
Epoch: [ 0] [   81/  468] time: 29.4881, train_loss: 0.13921148, train_accuracy: 0.9453, test_accuracy: 0.9271
Epoch: [ 0] [   82/  468] time: 29.8231, train_loss: 0.21368854, train_accuracy: 0.9297, test_accuracy: 0.9302
E

Epoch: [ 0] [  148/  468] time: 57.9570, train_loss: 0.18741757, train_accuracy: 0.9531, test_accuracy: 0.9219
Epoch: [ 0] [  149/  468] time: 58.3660, train_loss: 0.18300852, train_accuracy: 0.9609, test_accuracy: 0.9162
Epoch: [ 0] [  150/  468] time: 58.8320, train_loss: 0.29038677, train_accuracy: 0.9297, test_accuracy: 0.9361
Epoch: [ 0] [  151/  468] time: 59.2709, train_loss: 0.10267240, train_accuracy: 0.9766, test_accuracy: 0.9477
Epoch: [ 0] [  152/  468] time: 59.7769, train_loss: 0.18167557, train_accuracy: 0.9375, test_accuracy: 0.9455
Epoch: [ 0] [  153/  468] time: 60.2683, train_loss: 0.11834878, train_accuracy: 0.9688, test_accuracy: 0.9357
Epoch: [ 0] [  154/  468] time: 60.7253, train_loss: 0.23241022, train_accuracy: 0.9297, test_accuracy: 0.9282
Epoch: [ 0] [  155/  468] time: 61.2503, train_loss: 0.09848711, train_accuracy: 0.9844, test_accuracy: 0.9276
Epoch: [ 0] [  156/  468] time: 61.7082, train_loss: 0.14751966, train_accuracy: 0.9688, test_accuracy: 0.9337
E

Epoch: [ 0] [  222/  468] time: 91.8816, train_loss: 0.08709437, train_accuracy: 0.9688, test_accuracy: 0.9524
Epoch: [ 0] [  223/  468] time: 92.3156, train_loss: 0.07204066, train_accuracy: 0.9844, test_accuracy: 0.9586
Epoch: [ 0] [  224/  468] time: 92.8646, train_loss: 0.16498911, train_accuracy: 0.9453, test_accuracy: 0.9606
Epoch: [ 0] [  225/  468] time: 93.2385, train_loss: 0.10560912, train_accuracy: 0.9844, test_accuracy: 0.9576
Epoch: [ 0] [  226/  468] time: 93.6705, train_loss: 0.11274618, train_accuracy: 0.9609, test_accuracy: 0.9512
Epoch: [ 0] [  227/  468] time: 94.1247, train_loss: 0.14274342, train_accuracy: 0.9453, test_accuracy: 0.9497
Epoch: [ 0] [  228/  468] time: 94.5547, train_loss: 0.08543266, train_accuracy: 0.9844, test_accuracy: 0.9492
Epoch: [ 0] [  229/  468] time: 95.0017, train_loss: 0.09443536, train_accuracy: 0.9844, test_accuracy: 0.9490
Epoch: [ 0] [  230/  468] time: 95.6671, train_loss: 0.11038186, train_accuracy: 0.9531, test_accuracy: 0.9509
E

Epoch: [ 0] [  296/  468] time: 127.6929, train_loss: 0.05496098, train_accuracy: 0.9844, test_accuracy: 0.9525
Epoch: [ 0] [  297/  468] time: 128.2449, train_loss: 0.22789893, train_accuracy: 0.9375, test_accuracy: 0.9518
Epoch: [ 0] [  298/  468] time: 128.7509, train_loss: 0.06393886, train_accuracy: 0.9922, test_accuracy: 0.9537
Epoch: [ 0] [  299/  468] time: 129.2599, train_loss: 0.09759074, train_accuracy: 0.9766, test_accuracy: 0.9534
Epoch: [ 0] [  300/  468] time: 129.6829, train_loss: 0.14340621, train_accuracy: 0.9531, test_accuracy: 0.9583
Epoch: [ 0] [  301/  468] time: 130.1659, train_loss: 0.09838835, train_accuracy: 0.9688, test_accuracy: 0.9609
Epoch: [ 0] [  302/  468] time: 130.6328, train_loss: 0.02577091, train_accuracy: 0.9922, test_accuracy: 0.9599
Epoch: [ 0] [  303/  468] time: 131.0868, train_loss: 0.07163303, train_accuracy: 0.9922, test_accuracy: 0.9583
Epoch: [ 0] [  304/  468] time: 131.4881, train_loss: 0.15553880, train_accuracy: 0.9531, test_accuracy:

Epoch: [ 0] [  370/  468] time: 165.1353, train_loss: 0.14700812, train_accuracy: 0.9531, test_accuracy: 0.9564
Epoch: [ 0] [  371/  468] time: 165.6792, train_loss: 0.12501681, train_accuracy: 0.9688, test_accuracy: 0.9592
Epoch: [ 0] [  372/  468] time: 166.1632, train_loss: 0.09171091, train_accuracy: 0.9609, test_accuracy: 0.9632
Epoch: [ 0] [  373/  468] time: 166.6322, train_loss: 0.08394477, train_accuracy: 0.9844, test_accuracy: 0.9651
Epoch: [ 0] [  374/  468] time: 167.0982, train_loss: 0.11794709, train_accuracy: 0.9688, test_accuracy: 0.9647
Epoch: [ 0] [  375/  468] time: 167.6271, train_loss: 0.02590740, train_accuracy: 1.0000, test_accuracy: 0.9625
Epoch: [ 0] [  376/  468] time: 168.1429, train_loss: 0.06342869, train_accuracy: 0.9766, test_accuracy: 0.9612
Epoch: [ 0] [  377/  468] time: 168.6039, train_loss: 0.09324166, train_accuracy: 0.9766, test_accuracy: 0.9612
Epoch: [ 0] [  378/  468] time: 169.0519, train_loss: 0.08383031, train_accuracy: 0.9766, test_accuracy:

Epoch: [ 0] [  444/  468] time: 201.2188, train_loss: 0.22418478, train_accuracy: 0.9297, test_accuracy: 0.9518
Epoch: [ 0] [  445/  468] time: 201.6878, train_loss: 0.12040681, train_accuracy: 0.9609, test_accuracy: 0.9547
Epoch: [ 0] [  446/  468] time: 202.1458, train_loss: 0.03328840, train_accuracy: 0.9922, test_accuracy: 0.9604
Epoch: [ 0] [  447/  468] time: 202.6778, train_loss: 0.10336909, train_accuracy: 0.9688, test_accuracy: 0.9620
Epoch: [ 0] [  448/  468] time: 203.1458, train_loss: 0.06920445, train_accuracy: 0.9766, test_accuracy: 0.9642
Epoch: [ 0] [  449/  468] time: 203.6048, train_loss: 0.06274014, train_accuracy: 0.9844, test_accuracy: 0.9660
Epoch: [ 0] [  450/  468] time: 204.0743, train_loss: 0.07997546, train_accuracy: 0.9688, test_accuracy: 0.9666
Epoch: [ 0] [  451/  468] time: 204.7553, train_loss: 0.08408219, train_accuracy: 0.9688, test_accuracy: 0.9655
Epoch: [ 0] [  452/  468] time: 205.2313, train_loss: 0.07033139, train_accuracy: 0.9766, test_accuracy: